In [1]:
# Visualize MNIST dataset in Visdom
# Train MNIST
# Plot loss curve with Visdom

In [2]:
# First, open Terminal and type:

# python -m visdom.server

# Then, open browser and go to http://localhost:8097/

In [3]:
import torch

In [4]:
from torchvision.datasets import MNIST

In [5]:
from torchvision import transforms

In [6]:
# TODO: talk more about what these compose options do
transf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (1.0, ))])

In [7]:
train_set = MNIST(root='./data', train=True, transform=transf, download=False) # load training dataset

In [8]:
train_set.train_data.shape # see dataset size

torch.Size([60000, 28, 28])

torch.Size([60000, 28, 28])

In [9]:
train_set.train_data[0]

tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,

tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,

In [10]:
# To visualize the samples in dataset, let's use Visdom
from visdom import Visdom
vis = Visdom()

In [11]:
vis.image(train_set.train_data[0]) # See image in Visdom page

'window_3629878c6dce0a'

'window_3629878c6dce0a'

In [12]:
train_set.train_labels[0]

tensor(5)

tensor(5)

In [13]:
test_set = MNIST(root='./data', train=False, transform=transf, download=False) # load training dataset

In [14]:
from torchvision.utils import make_grid # Combine images into a nice-looking grid layout

In [15]:
# first 10 samples combined together
# MNIST dataset doesn't have channel dimension, so we use unsqueeze to manually add it
print(train_set.train_data.shape)
print(train_set.train_data.unsqueeze(1).shape)
combined_image = make_grid(train_set.train_data[:10].unsqueeze(1))

torch.Size([60000, 28, 28])
torch.Size([60000, 1, 28, 28])
torch.Size([60000, 28, 28])
torch.Size([60000, 1, 28, 28])


In [16]:
vis.image(combined_image)

'window_3629878c7e7a50'

'window_3629878c7e7a50'

In [17]:
# Better way to visualize how good our model does
# For each target value (0-9), see how often our model gets it right
def confusion(model, n, dataset):
    conf = torch.zeros(n, n)
    model.eval() # Put model in evaluation mode
    for data, target in dataset:
        data = data.unsqueeze(1)
        output = model(data)
        _, pred = torch.max(output.data, 1)
        conf[target][pred[0]] += 1
        
    # Normalize
    for i in range(n):
        conf[i] = conf[i] / conf[i].sum()
        
    vis.image(conf)
    
# A perfect model should show a diagonal white line

In [18]:
import torch.nn as nn
import torch.nn.functional as F

In [19]:
# Define our own CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(20 * 4 * 4, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        # we can also use functional interface for Conv2d and pass in the weights and biases, useful in weight sharing scenario
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 20 * 4 * 4) # -1 means we only care about the size of the last dimension, and just merge the rest of the dimensions into one
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [20]:
from torch import optim

In [21]:
from torch.optim import lr_scheduler

In [22]:
from torch.utils.data import TensorDataset, DataLoader

In [23]:
def train_model(model, train_set, test_set, nepochs):
    batch_size = 100
    # dataloader can help you get batched data and shuffle the data
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set)
    
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
    
    model.train()
    
    for epoch in range(nepochs):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad() # This will zero out all the gradients for the model's parameters
            output = model(data) # This is a softmax output over the possible labels
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), float(loss)))

        model.eval()
        test_loss = 0.0
        correct = 0
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).data.item()
            
            _, pred = torch.max(output.data, 1)
            
            correct += (pred == target.data).sum()
            
        test_loss /= len(test_set)
        print("Loss: ", test_loss, "Accuracy: ", float(correct)/len(test_set)) # TODO: use Visdom to plot the loss curve

In [24]:
model = CNN()

In [25]:
train_model(model, train_set, test_set, 10)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.301549
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.301549
Train Epoch: 0 [1000/60000 (2%)]	Loss: 2.293043
Train Epoch: 0 [1000/60000 (2%)]	Loss: 2.293043
Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.301735
Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.301735
Train Epoch: 0 [3000/60000 (5%)]	Loss: 2.289609
Train Epoch: 0 [3000/60000 (5%)]	Loss: 2.289609
Train Epoch: 0 [4000/60000 (7%)]	Loss: 2.296005
Train Epoch: 0 [4000/60000 (7%)]	Loss: 2.296005
Train Epoch: 0 [5000/60000 (8%)]	Loss: 2.278799
Train Epoch: 0 [5000/60000 (8%)]	Loss: 2.278799
Train Epoch: 0 [6000/60000 (10%)]	Loss: 2.284930
Train Epoch: 0 [6000/60000 (10%)]	Loss: 2.284930
Train Epoch: 0 [7000/60000 (12%)]	Loss: 2.291004
Train Epoch: 0 [7000/60000 (12%)]	Loss: 2.291004
Train Epoch: 0 [8000/60000 (13%)]	Loss: 2.273129
Train Epoch: 0 [8000/60000 (13%)]	Loss: 2.273129
Train Epoch: 0 [9000/60000 (15%)]	Loss: 2.273779
Train Epoch: 0 [9000/60000 (15%)]	Loss: 2.273779
Train Epoch: 0 [10000/60000 (17%)]	Los

Train Epoch: 1 [22000/60000 (37%)]	Loss: 0.200707
Train Epoch: 1 [22000/60000 (37%)]	Loss: 0.200707
Train Epoch: 1 [23000/60000 (38%)]	Loss: 0.263717
Train Epoch: 1 [23000/60000 (38%)]	Loss: 0.263717
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.223092
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.223092
Train Epoch: 1 [25000/60000 (42%)]	Loss: 0.273759
Train Epoch: 1 [25000/60000 (42%)]	Loss: 0.273759
Train Epoch: 1 [26000/60000 (43%)]	Loss: 0.171702
Train Epoch: 1 [26000/60000 (43%)]	Loss: 0.171702
Train Epoch: 1 [27000/60000 (45%)]	Loss: 0.222628
Train Epoch: 1 [27000/60000 (45%)]	Loss: 0.222628
Train Epoch: 1 [28000/60000 (47%)]	Loss: 0.285842
Train Epoch: 1 [28000/60000 (47%)]	Loss: 0.285842
Train Epoch: 1 [29000/60000 (48%)]	Loss: 0.194934
Train Epoch: 1 [29000/60000 (48%)]	Loss: 0.194934
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.088165
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.088165
Train Epoch: 1 [31000/60000 (52%)]	Loss: 0.114989
Train Epoch: 1 [31000/60000 (52%)]	Loss: 0.114989


Train Epoch: 2 [43000/60000 (72%)]	Loss: 0.087072
Train Epoch: 2 [44000/60000 (73%)]	Loss: 0.176538
Train Epoch: 2 [44000/60000 (73%)]	Loss: 0.176538
Train Epoch: 2 [45000/60000 (75%)]	Loss: 0.163603
Train Epoch: 2 [45000/60000 (75%)]	Loss: 0.163603
Train Epoch: 2 [46000/60000 (77%)]	Loss: 0.125265
Train Epoch: 2 [46000/60000 (77%)]	Loss: 0.125265
Train Epoch: 2 [47000/60000 (78%)]	Loss: 0.120685
Train Epoch: 2 [47000/60000 (78%)]	Loss: 0.120685
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.123434
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.123434
Train Epoch: 2 [49000/60000 (82%)]	Loss: 0.088034
Train Epoch: 2 [49000/60000 (82%)]	Loss: 0.088034
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.109236
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.109236
Train Epoch: 2 [51000/60000 (85%)]	Loss: 0.171704
Train Epoch: 2 [51000/60000 (85%)]	Loss: 0.171704
Train Epoch: 2 [52000/60000 (87%)]	Loss: 0.166031
Train Epoch: 2 [52000/60000 (87%)]	Loss: 0.166031
Train Epoch: 2 [53000/60000 (88%)]	Loss: 0.082691


Train Epoch: 4 [4000/60000 (7%)]	Loss: 0.078346
Train Epoch: 4 [5000/60000 (8%)]	Loss: 0.093604
Train Epoch: 4 [5000/60000 (8%)]	Loss: 0.093604
Train Epoch: 4 [6000/60000 (10%)]	Loss: 0.089024
Train Epoch: 4 [6000/60000 (10%)]	Loss: 0.089024
Train Epoch: 4 [7000/60000 (12%)]	Loss: 0.092476
Train Epoch: 4 [7000/60000 (12%)]	Loss: 0.092476
Train Epoch: 4 [8000/60000 (13%)]	Loss: 0.177079
Train Epoch: 4 [8000/60000 (13%)]	Loss: 0.177079
Train Epoch: 4 [9000/60000 (15%)]	Loss: 0.063722
Train Epoch: 4 [9000/60000 (15%)]	Loss: 0.063722
Train Epoch: 4 [10000/60000 (17%)]	Loss: 0.199708
Train Epoch: 4 [10000/60000 (17%)]	Loss: 0.199708
Train Epoch: 4 [11000/60000 (18%)]	Loss: 0.024100
Train Epoch: 4 [11000/60000 (18%)]	Loss: 0.024100
Train Epoch: 4 [12000/60000 (20%)]	Loss: 0.105219
Train Epoch: 4 [12000/60000 (20%)]	Loss: 0.105219
Train Epoch: 4 [13000/60000 (22%)]	Loss: 0.085196
Train Epoch: 4 [13000/60000 (22%)]	Loss: 0.085196
Train Epoch: 4 [14000/60000 (23%)]	Loss: 0.090868
Train Epoch: 4

Train Epoch: 5 [26000/60000 (43%)]	Loss: 0.073080
Train Epoch: 5 [27000/60000 (45%)]	Loss: 0.058706
Train Epoch: 5 [27000/60000 (45%)]	Loss: 0.058706
Train Epoch: 5 [28000/60000 (47%)]	Loss: 0.058766
Train Epoch: 5 [28000/60000 (47%)]	Loss: 0.058766
Train Epoch: 5 [29000/60000 (48%)]	Loss: 0.070153
Train Epoch: 5 [29000/60000 (48%)]	Loss: 0.070153
Train Epoch: 5 [30000/60000 (50%)]	Loss: 0.129190
Train Epoch: 5 [30000/60000 (50%)]	Loss: 0.129190
Train Epoch: 5 [31000/60000 (52%)]	Loss: 0.113430
Train Epoch: 5 [31000/60000 (52%)]	Loss: 0.113430
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.092909
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.092909
Train Epoch: 5 [33000/60000 (55%)]	Loss: 0.061588
Train Epoch: 5 [33000/60000 (55%)]	Loss: 0.061588
Train Epoch: 5 [34000/60000 (57%)]	Loss: 0.077016
Train Epoch: 5 [34000/60000 (57%)]	Loss: 0.077016
Train Epoch: 5 [35000/60000 (58%)]	Loss: 0.073172
Train Epoch: 5 [35000/60000 (58%)]	Loss: 0.073172
Train Epoch: 5 [36000/60000 (60%)]	Loss: 0.052266


Train Epoch: 6 [48000/60000 (80%)]	Loss: 0.055431
Train Epoch: 6 [48000/60000 (80%)]	Loss: 0.055431
Train Epoch: 6 [49000/60000 (82%)]	Loss: 0.118359
Train Epoch: 6 [49000/60000 (82%)]	Loss: 0.118359
Train Epoch: 6 [50000/60000 (83%)]	Loss: 0.120248
Train Epoch: 6 [50000/60000 (83%)]	Loss: 0.120248
Train Epoch: 6 [51000/60000 (85%)]	Loss: 0.027362
Train Epoch: 6 [51000/60000 (85%)]	Loss: 0.027362
Train Epoch: 6 [52000/60000 (87%)]	Loss: 0.127269
Train Epoch: 6 [52000/60000 (87%)]	Loss: 0.127269
Train Epoch: 6 [53000/60000 (88%)]	Loss: 0.049001
Train Epoch: 6 [53000/60000 (88%)]	Loss: 0.049001
Train Epoch: 6 [54000/60000 (90%)]	Loss: 0.053380
Train Epoch: 6 [54000/60000 (90%)]	Loss: 0.053380
Train Epoch: 6 [55000/60000 (92%)]	Loss: 0.017229
Train Epoch: 6 [55000/60000 (92%)]	Loss: 0.017229
Train Epoch: 6 [56000/60000 (93%)]	Loss: 0.086372
Train Epoch: 6 [56000/60000 (93%)]	Loss: 0.086372
Train Epoch: 6 [57000/60000 (95%)]	Loss: 0.063392
Train Epoch: 6 [57000/60000 (95%)]	Loss: 0.063392


Train Epoch: 8 [9000/60000 (15%)]	Loss: 0.034012
Train Epoch: 8 [9000/60000 (15%)]	Loss: 0.034012
Train Epoch: 8 [10000/60000 (17%)]	Loss: 0.081175
Train Epoch: 8 [10000/60000 (17%)]	Loss: 0.081175
Train Epoch: 8 [11000/60000 (18%)]	Loss: 0.103123
Train Epoch: 8 [11000/60000 (18%)]	Loss: 0.103123
Train Epoch: 8 [12000/60000 (20%)]	Loss: 0.027646
Train Epoch: 8 [12000/60000 (20%)]	Loss: 0.027646
Train Epoch: 8 [13000/60000 (22%)]	Loss: 0.092255
Train Epoch: 8 [13000/60000 (22%)]	Loss: 0.092255
Train Epoch: 8 [14000/60000 (23%)]	Loss: 0.035138
Train Epoch: 8 [14000/60000 (23%)]	Loss: 0.035138
Train Epoch: 8 [15000/60000 (25%)]	Loss: 0.018377
Train Epoch: 8 [15000/60000 (25%)]	Loss: 0.018377
Train Epoch: 8 [16000/60000 (27%)]	Loss: 0.101556
Train Epoch: 8 [16000/60000 (27%)]	Loss: 0.101556
Train Epoch: 8 [17000/60000 (28%)]	Loss: 0.077851
Train Epoch: 8 [17000/60000 (28%)]	Loss: 0.077851
Train Epoch: 8 [18000/60000 (30%)]	Loss: 0.052268
Train Epoch: 8 [18000/60000 (30%)]	Loss: 0.052268
Tr

Train Epoch: 9 [30000/60000 (50%)]	Loss: 0.045028
Train Epoch: 9 [31000/60000 (52%)]	Loss: 0.023339
Train Epoch: 9 [31000/60000 (52%)]	Loss: 0.023339
Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.044974
Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.044974
Train Epoch: 9 [33000/60000 (55%)]	Loss: 0.064926
Train Epoch: 9 [33000/60000 (55%)]	Loss: 0.064926
Train Epoch: 9 [34000/60000 (57%)]	Loss: 0.025661
Train Epoch: 9 [34000/60000 (57%)]	Loss: 0.025661
Train Epoch: 9 [35000/60000 (58%)]	Loss: 0.058612
Train Epoch: 9 [35000/60000 (58%)]	Loss: 0.058612
Train Epoch: 9 [36000/60000 (60%)]	Loss: 0.048961
Train Epoch: 9 [36000/60000 (60%)]	Loss: 0.048961
Train Epoch: 9 [37000/60000 (62%)]	Loss: 0.027891
Train Epoch: 9 [37000/60000 (62%)]	Loss: 0.027891
Train Epoch: 9 [38000/60000 (63%)]	Loss: 0.084607
Train Epoch: 9 [38000/60000 (63%)]	Loss: 0.084607
Train Epoch: 9 [39000/60000 (65%)]	Loss: 0.043557
Train Epoch: 9 [39000/60000 (65%)]	Loss: 0.043557
Train Epoch: 9 [40000/60000 (67%)]	Loss: 0.008238


In [28]:
confusion(model, 10, test_set) # See confusion matrix

In [29]:
# Save model state as checkpoint, to be used later
torch.save(model.state_dict(), 'mnist.pth')

# Load model into another variable
new_model = CNN()
new_model.load_state_dict(torch.load('mnist.pth'))

In [30]:
new_model

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [31]:
new_model

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [32]:
model

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
# TODO: for transfer learning, make sure the two linear layers are replaced